In [1]:
import pandas as pd
import pdb

from utils.experiment_utils import *
""
%load_ext autoreload
%autoreload 2

Read in all saved results and make Latex table

Metrics:
* marginal cov
* ACCG
* average set size


To get multiple row levels, set a multi-index. To get multiple column levels, concatenate dataframes along axis=1.

In [50]:
# Map from dataset to folder with saved results
# datasets = {'ImageNet': '.cache/paper/imagenet',
#             'CIFAR-100': '.cache/paper/cifar100',
#             'Places365': '.cache/paper/places365',
#             'iNaturalist': '.cache/paper/inaturalist'}
datasets = {'ImageNet': '.cache/paper/varying_n/imagenet',
            'CIFAR-100': '.cache/paper/varying_n/cifar-100',
            'Places365': '.cache/paper/varying_n/places365',
            'iNaturalist': '.cache/paper/varying_n/inaturalist'}

score_functions = {'softmax': '$\softmax$',
                  'APS': '$\APS$',
                  'RAPS': '$\RAPS$'}

methods = {'standard': '$\standard$', 
            'classwise': '$\classwise$',
#             'cluster_proportional': '$\clustered$',
            'cluster_random': '$\clustered$'} # TODO: update

calibration_sampling = 'random' # 'random' or 'balanced'

In [51]:
# # GENERATE TABLE - two n's and including marginal coverage
# n_totalcals = {10: '$n=10$', 50: '$n=50$'}

# horizontal_dict = {}
# for n_totalcal, n_name in n_totalcals.items():
#     vertical_list = []
#     for dataset, save_folder in datasets.items():
#         for score, score_name in score_functions.items():
#             for method, method_name in methods.items(): 

#                 folder = f'{save_folder}/{calibration_sampling}_calset/n_totalcal={n_totalcal}/score={score}/'
#                 print(folder)
#                 df = average_results_across_seeds(folder, print_results=False, methods=methods.keys()) # df = results DataFrame
#                 s = pd.Series({'Dataset': dataset, 'Score func.': score_name, 'Method': method_name, 
#                                'CovGap': f"{df[df['method'] == method]['class_cov_gap_mean'].values[0]:.1f}" 
#                                        f" ({df[df['method'] == method]['class_cov_gap_se'].values[0]:.1f})",
#                                'AvgSize': f"{df[df['method'] == method]['avg_set_size_mean'].values[0]:.1f}" 
#                                                f" ({df[df['method'] == method]['avg_set_size_se'].values[0]:.1f})",
#                                'MargCov': f"{df[df['method'] == method]['marginal_cov_mean'].values[0]*100:.1f}" 
#                                                    f" ({df[df['method'] == method]['marginal_cov_se'].values[0]*100:.1f})"})
#                 vertical_list.append(pd.DataFrame(s).transpose())                                        

#     df = pd.concat(vertical_list, ignore_index=True)                                                     
#     df = df.set_index(['Dataset', 'Score func.', 'Method'])
#     horizontal_dict[n_name] = df
# #     pdb.set_trace()

# df = pd.concat(horizontal_dict, axis=1)
# df_str = df.to_latex(multicolumn_format='c', # To center multilevel column headers
#     escape=False # So that backslashes are printed as `\' rather than '\textbackslash'
#                  )
# # Post-process
# df_str = df_str.replace('lllllllll', 'lllcrccrc')
# df_str = df_str.replace('\multicolumn{3}{c}{$n=10$} & \multicolumn{3}{c}{$n=50$} \\',
#                '\multicolumn{3}{c}{$n=10$} & \multicolumn{3}{c}{$n=50$} \\\\ \cmidrule(lr){4-6}\cmidrule(lr){7-9} \\')
# df_str = df_str.replace('CIFAR-100', '\midrule\nCIFAR-100')
# df_str = df_str.replace('Places365', '\midrule\nPlaces365')
# df_str = df_str.replace('iNaturalist', '\midrule\niNaturalist')
# print(df_str)

In [60]:
# GENERATE TABLE - three n's and excluding marginal coverage
n_totalcals = {10: '$n_{\text{avg}}=10$', 20: '$n_{\text{avg}}=20$', 
               50: '$n_{\text{avg}}=50$', 75: '$n_{\text{avg}}=75$'}

horizontal_dict = {}
for n_totalcal, n_name in n_totalcals.items():
    vertical_list = []
    for dataset, save_folder in datasets.items():
        for score, score_name in score_functions.items():
            for method, method_name in methods.items(): 

                folder = f'{save_folder}/{calibration_sampling}_calset/n_totalcal={n_totalcal}/score={score}/'
                print(folder)
                df = average_results_across_seeds(folder, print_results=False, methods=methods.keys()) # df = results DataFrame
                s = pd.Series({'Dataset': dataset, 'Score func.': score_name, 'Method': method_name, 
                               'CovGap': f"{df[df['method'] == method]['class_cov_gap_mean'].values[0]:.1f}" 
                                       f" ({df[df['method'] == method]['class_cov_gap_se'].values[0]:.1f})",
                               'AvgSize': f"{df[df['method'] == method]['avg_set_size_mean'].values[0]:.1f}" 
                                               f" ({df[df['method'] == method]['avg_set_size_se'].values[0]:.1f})"})
#                                'MargCov': f"{df[df['method'] == method]['marginal_cov_mean'].values[0]*100:.1f}" 
#                                                    f" ({df[df['method'] == method]['marginal_cov_se'].values[0]*100:.1f})"})
                vertical_list.append(pd.DataFrame(s).transpose())                                        

    df = pd.concat(vertical_list, ignore_index=True)                                                     
    df = df.set_index(['Dataset', 'Score func.', 'Method'])
    horizontal_dict[n_name] = df
#     pdb.set_trace()

df = pd.concat(horizontal_dict, axis=1)
df_str = df.to_latex(multicolumn_format='c', # To center multilevel column headers
    escape=False # So that backslashes are printed as `\' rather than '\textbackslash'
                 )
# Post-process
df_str = df_str.replace('lllllllllll', 'lllcrcrcrcr')
df_str = df_str.replace('\\multicolumn{2}{c}{$n_{\\text{avg}}=75$} \\\\',
               '\\multicolumn{2}{c}{$n_{\\text{avg}}=75$} \\\\ \cmidrule(lr){4-5}\cmidrule(lr){6-7} \cmidrule(lr){8-9} \cmidrule(lr){10-11}\\')
df_str = df_str.replace('CIFAR-100', '\midrule\nCIFAR-100')
df_str = df_str.replace('Places365', '\midrule\nPlaces365')
df_str = df_str.replace('iNaturalist', '\midrule\niNaturalist')
# Add horizontal lines between methods
df_str = df_str.replace('& $\APS$', '\cline{3-11} & $\APS$')
df_str = df_str.replace('& $\RAPS$', '\cline{3-11} & $\RAPS$')
print(df_str)

.cache/paper/varying_n/imagenet/random_calset/n_totalcal=10/score=softmax/
Number of seeds found: 10


,method,class_cov_gap_mean,class_cov_gap_se,max_class_cov_gap_mean,max_class_cov_gap_se,avg_set_size_mean,avg_set_size_se,marginal_cov_mean,marginal_cov_se,very_undercovered_mean,very_undercovered_se
0,standard,5.223009,0.026946,38.294309,0.320930,1.940409,0.017132,0.900390,0.000849,0.0784,0.003021
1,classwise,7.665144,0.045728,38.259073,1.171264,354.400610,2.023354,0.946082,0.000518,0.0547,0.001744
2,cluster_random,4.871419,0.087259,34.997894,1.532120,2.500469,0.121130,0.903487,0.002734,0.0682,0.008114


.cache/paper/varying_n/imagenet/random_calset/n_totalcal=10/score=softmax/
Number of seeds found: 10


,method,class_cov_gap_mean,class_cov_gap_se,max_class_cov_gap_mean,max_class_cov_gap_se,avg_set_size_mean,avg_set_size_se,marginal_cov_mean,marginal_cov_se,very_undercovered_mean,very_undercovered_se
0,standard,5.223009,0.026946,38.294309,0.320930,1.940409,0.017132,0.900390,0.000849,0.0784,0.003021
1,classwise,7.665144,0.045728,38.259073,1.171264,354.400610,2.023354,0.946082,0.000518,0.0547,0.001744
2,cluster_random,4.871419,0.087259,34.997894,1.532120,2.500469,0.121130,0.903487,0.002734,0.0682,0.008114


.cache/paper/varying_n/imagenet/random_calset/n_totalcal=10/score=softmax/
Number of seeds found: 10


...

,method,class_cov_gap_mean,class_cov_gap_se,max_class_cov_gap_mean,max_class_cov_gap_se,avg_set_size_mean,avg_set_size_se,marginal_cov_mean,marginal_cov_se,very_undercovered_mean,very_undercovered_se
0,standard,4.962075,0.018908,56.892585,0.233436,5.158425,0.011148,0.899221,0.000467,0.127488,0.001323
1,classwise,4.856312,0.043951,27.448262,1.082188,63.111438,1.672252,0.906876,0.000483,0.032385,0.002134
2,cluster_random,4.104375,0.054755,57.146218,0.295667,5.424337,0.057444,0.897728,0.000859,0.082780,0.002819


.cache/paper/varying_n/inaturalist/random_calset/n_totalcal=75/score=RAPS/
Number of seeds found: 10


,method,class_cov_gap_mean,class_cov_gap_se,max_class_cov_gap_mean,max_class_cov_gap_se,avg_set_size_mean,avg_set_size_se,marginal_cov_mean,marginal_cov_se,very_undercovered_mean,very_undercovered_se
0,standard,4.962075,0.018908,56.892585,0.233436,5.158425,0.011148,0.899221,0.000467,0.127488,0.001323
1,classwise,4.856312,0.043951,27.448262,1.082188,63.111438,1.672252,0.906876,0.000483,0.032385,0.002134
2,cluster_random,4.104375,0.054755,57.146218,0.295667,5.424337,0.057444,0.897728,0.000859,0.082780,0.002819


\begin{tabular}{lllcrcrcrcr}
\toprule
            &         &              & \multicolumn{2}{c}{$n_{\text{avg}}=10$} & \multicolumn{2}{c}{$n_{\text{avg}}=20$} & \multicolumn{2}{c}{$n_{\text{avg}}=50$} & \multicolumn{2}{c}{$n_{\text{avg}}=75$} \\ \cmidrule(lr){4-5}\cmidrule(lr){6-7} \cmidrule(lr){8-9} \cmidrule(lr){10-11}\
            &         &              &              CovGap &      AvgSize &              CovGap &      AvgSize &              CovGap &      AvgSize &              CovGap &     AvgSize \\
Dataset & Score func. & Method &                     &              &                     &              &                     &              &                     &             \\
\midrule
ImageNet & $\softmax$ & $\standard$ &           5.2 (0.0) &    1.9 (0.0) &           5.2 (0.0) &    1.9 (0.0) &           5.3 (0.0) &    1.9 (0.0) &           5.2 (0.0) &   1.9 (0.0) \\
            &         & $\classwise$ &           7.7 (0.0) &  354.4 (2.0) &           5.4 (0.0) &   23.0 (0.9) & 

/tmp/ipykernel_1785434/3021232834.py:30: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df_str = df.to_latex(multicolumn_format='c', # To center multilevel column headers


In [54]:
# Investigate marginal coverage
n_totalcals = {10: '$n_{\text{avg}}=10$', 30: '$n_{\text{avg}}=30$', 75: '$n_{\text{avg}}=75$'}

horizontal_dict = {}
for n_totalcal, n_name in n_totalcals.items():
    vertical_list = []
    for dataset, save_folder in datasets.items():
        for score, score_name in score_functions.items():
            for method, method_name in methods.items(): 

                folder = f'{save_folder}/{calibration_sampling}_calset/n_totalcal={n_totalcal}/score={score}/'
                print(folder)
                df = average_results_across_seeds(folder, print_results=False, methods=methods.keys()) # df = results DataFrame
                s = pd.Series({'Dataset': dataset, 'Score func.': score_name, 'Method': method_name, 
#                                'CovGap': f"{df[df['method'] == method]['class_cov_gap_mean'].values[0]:.1f}" 
#                                        f" ({df[df['method'] == method]['class_cov_gap_se'].values[0]:.1f})",
#                                'AvgSize': f"{df[df['method'] == method]['avg_set_size_mean'].values[0]:.1f}" 
#                                                f" ({df[df['method'] == method]['avg_set_size_se'].values[0]:.1f})",
                               'MargCov': f"{df[df['method'] == method]['marginal_cov_mean'].values[0]*100:.1f}" 
                                                   f" ({df[df['method'] == method]['marginal_cov_se'].values[0]*100:.1f})"})
                vertical_list.append(pd.DataFrame(s).transpose())                                        

    df = pd.concat(vertical_list, ignore_index=True)                                                     
    df = df.set_index(['Dataset', 'Score func.', 'Method'])
    horizontal_dict[n_name] = df
#     pdb.set_trace()

df = pd.concat(horizontal_dict, axis=1)
df_str = df.to_latex(multicolumn_format='c', # To center multilevel column headers
    escape=False # So that backslashes are printed as `\' rather than '\textbackslash'
                 )
# Post-process
# df_str = df_str.replace('lllllllll', 'lllcrccrc')
# df_str = df_str.replace('$n_{\\text{avg}}=75$ \\',
#                '$n_{\\text{avg}}=75$ \\\\ \cmidrule(lr){4}\cmidrule(lr){5} \cmidrule(lr){6}\\')
df_str = df_str.replace('CIFAR-100', '\midrule\nCIFAR-100')
df_str = df_str.replace('Places365', '\midrule\nPlaces365')
df_str = df_str.replace('iNaturalist', '\midrule\niNaturalist')
print(df_str)

.cache/paper/varying_n/imagenet/random_calset/n_totalcal=10/score=softmax/
Number of seeds found: 10


,method,class_cov_gap_mean,class_cov_gap_se,max_class_cov_gap_mean,max_class_cov_gap_se,avg_set_size_mean,avg_set_size_se,marginal_cov_mean,marginal_cov_se,very_undercovered_mean,very_undercovered_se
0,standard,5.223009,0.026946,38.294309,0.320930,1.940409,0.017132,0.900390,0.000849,0.0784,0.003021
1,classwise,7.665144,0.045728,38.259073,1.171264,354.400610,2.023354,0.946082,0.000518,0.0547,0.001744
2,cluster_random,4.871419,0.087259,34.997894,1.532120,2.500469,0.121130,0.903487,0.002734,0.0682,0.008114


.cache/paper/varying_n/imagenet/random_calset/n_totalcal=10/score=softmax/
Number of seeds found: 10


,method,class_cov_gap_mean,class_cov_gap_se,max_class_cov_gap_mean,max_class_cov_gap_se,avg_set_size_mean,avg_set_size_se,marginal_cov_mean,marginal_cov_se,very_undercovered_mean,very_undercovered_se
0,standard,5.223009,0.026946,38.294309,0.320930,1.940409,0.017132,0.900390,0.000849,0.0784,0.003021
1,classwise,7.665144,0.045728,38.259073,1.171264,354.400610,2.023354,0.946082,0.000518,0.0547,0.001744
2,cluster_random,4.871419,0.087259,34.997894,1.532120,2.500469,0.121130,0.903487,0.002734,0.0682,0.008114


.cache/paper/varying_n/imagenet/random_calset/n_totalcal=10/score=softmax/
Number of seeds found: 10


...

,method,class_cov_gap_mean,class_cov_gap_se,max_class_cov_gap_mean,max_class_cov_gap_se,avg_set_size_mean,avg_set_size_se,marginal_cov_mean,marginal_cov_se,very_undercovered_mean,very_undercovered_se
0,standard,4.962075,0.018908,56.892585,0.233436,5.158425,0.011148,0.899221,0.000467,0.127488,0.001323
1,classwise,4.856312,0.043951,27.448262,1.082188,63.111438,1.672252,0.906876,0.000483,0.032385,0.002134
2,cluster_random,4.104375,0.054755,57.146218,0.295667,5.424337,0.057444,0.897728,0.000859,0.082780,0.002819


.cache/paper/varying_n/inaturalist/random_calset/n_totalcal=75/score=RAPS/
Number of seeds found: 10


,method,class_cov_gap_mean,class_cov_gap_se,max_class_cov_gap_mean,max_class_cov_gap_se,avg_set_size_mean,avg_set_size_se,marginal_cov_mean,marginal_cov_se,very_undercovered_mean,very_undercovered_se
0,standard,4.962075,0.018908,56.892585,0.233436,5.158425,0.011148,0.899221,0.000467,0.127488,0.001323
1,classwise,4.856312,0.043951,27.448262,1.082188,63.111438,1.672252,0.906876,0.000483,0.032385,0.002134
2,cluster_random,4.104375,0.054755,57.146218,0.295667,5.424337,0.057444,0.897728,0.000859,0.082780,0.002819


\begin{tabular}{llllll}
\toprule
            &         &              & $n_{\text{avg}}=10$ & $n_{\text{avg}}=30$ & $n_{\text{avg}}=75$ \\
            &         &              &             MargCov &             MargCov &             MargCov \\
Dataset & Score func. & Method &                     &                     &                     \\
\midrule
ImageNet & $\softmax$ & $\standard$ &          90.0 (0.1) &          89.9 (0.0) &          89.9 (0.0) \\
            &         & $\classwise$ &          94.6 (0.1) &          91.6 (0.0) &          90.6 (0.0) \\
            &         & $\clustered$ &          90.3 (0.3) &          90.2 (0.2) &          90.1 (0.1) \\
            & $\APS$ & $\standard$ &          90.0 (0.1) &          89.9 (0.0) &          90.0 (0.0) \\
            &         & $\classwise$ &          94.7 (0.1) &          91.6 (0.1) &          90.7 (0.0) \\
            &         & $\clustered$ &          90.4 (0.3) &          90.1 (0.2) &          90.1 (0.1) \\
            &

/tmp/ipykernel_1785434/3360529266.py:29: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df_str = df.to_latex(multicolumn_format='c', # To center multilevel column headers


In [29]:
df

$n_{\text{avg}}=10$ $n_{\text{avg}}=30$  \
                                                 MargCov             MargCov   
Dataset     Score func. Method                                                 
ImageNet    $\softmax$  $\standard$           90.0 (0.1)          89.9 (0.0)   
                        $\classwise$          94.6 (0.1)          91.6 (0.0)   
                        $\clustered$          90.4 (0.2)          89.9 (0.1)   
            $\APS$      $\standard$           90.0 (0.1)          89.9 (0.0)   
                        $\classwise$          94.7 (0.1)          91.6 (0.1)   
                        $\clustered$          90.3 (0.1)          90.1 (0.2)   
            $\RAPS$     $\standard$           90.1 (0.1)          90.0 (0.1)   
                        $\classwise$          94.6 (0.1)          91.9 (0.1)   
                        $\clustered$          90.6 (0.2)          90.0 (0.2)   
CIFAR-100   $\softmax$  $\standard$           90.0 (0.1)          89.9 (0.1)   
                        $\classwise$          94.5 (0.1)          91.5 (0.1)   
                        $\clustered$          89.9 (0.2)          89.9 (0.1)   
            $\APS$      $\standard$           89.9 (0.1)          89.8 (0.1)   
                        $\classwise$          94.5 (0.1)          91.5 (0.1)   
                        $\clustered$          89.9 (0.2)          90.0 (0.1)   
            $\RAPS$     $\standard$           90.1 (0.1)          90.0 (0.1)   
                        $\classwise$          94.5 (0.1)          91.9 (0.1)   
                        $\clustered$          90.2 (0.1)          90.2 (0.1)   
Places365   $\softmax$  $\standard$           90.1 (0.1)          90.1 (0.1)   
                        $\classwise$          94.5 (0.1)          91.7 (0.1)   
                        $\clustered$          89.9 (0.3)          90.3 (0.1)   
            $\APS$      $\standard$           90.1 (0.1)          90.1 (0.1)   
                        $\classwise$          94.5 (0.1)          91.6 (0.1)   
                        $\clustered$          89.8 (0.4)          90.4 (0.1)   
            $\RAPS$     $\standard$           89.9 (0.1)          90.0 (0.1)   
                        $\classwise$          94.5 (0.1)          91.8 (0.1)   
                        $\clustered$          89.9 (0.3)          90.4 (0.1)   
iNaturalist $\softmax$  $\standard$           89.9 (0.1)          90.0 (0.1)   
                        $\classwise$          93.2 (0.1)          91.6 (0.0)   
                        $\clustered$          89.1 (0.2)          89.6 (0.1)   
            $\APS$      $\standard$           89.9 (0.2)          89.9 (0.1)   
                        $\classwise$          93.4 (0.1)          91.5 (0.1)   
                        $\clustered$          89.2 (0.2)          89.6 (0.1)   
            $\RAPS$     $\standard$           89.8 (0.2)          89.8 (0.1)   
                        $\classwise$          93.4 (0.1)          91.6 (0.1)   
                        $\clustered$          89.1 (0.2)          89.6 (0.1)   

                                     $n_{\text{avg}}=75$  
                                                 MargCov  
Dataset     Score func. Method                            
ImageNet    $\softmax$  $\standard$           89.9 (0.0)  
                        $\classwise$          90.6 (0.0)  
                        $\clustered$          90.3 (0.1)  
            $\APS$      $\standard$           90.0 (0.0)  
                        $\classwise$          90.7 (0.0)  
                        $\clustered$          90.2 (0.1)  
            $\RAPS$     $\standard$           90.0 (0.0)  
                        $\classwise$          90.7 (0.0)  
                        $\clustered$          90.2 (0.1)  
CIFAR-100   $\softmax$  $\standard$           90.0 (0.1)  
                        $\classwise$          90.7 (0.0)  
                        $\clustered$          90.3 (0.1)  
            $\APS$      $\stand